In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch 
from torch import nn
from torch.utils.data import Dataset, DataLoader

from scipy import ndimage as ndi
from scipy import fft, ifft
import scipy

from skimage import data
from skimage import io, transform, color
from skimage.util import img_as_float, pad
from skimage.filters import gabor_kernel

import pickle 
import os
import sys
import h5py
import warnings
import time
import json
import nibabel as nib

import gfb_utils as g
import nsd_utils as n

sys.path.append('/user_data/mmhender/coco_annot/cocoapi/PythonAPI/')
from pycocotools.coco import COCO


In [2]:
# Define directories and what sessions/subjects to load

code_dir = '/user_data/mmhender/test_code/'
nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/'

trials_per_sess = 750
nsess = 40
trials_per_subj = trials_per_sess*nsess
subj = 1
sess = 3

subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
subj_roi_dir = os.path.join(nsd_dir, 'nsddata','ppdata','subj%02d'%subj,
                           'func1pt8mm','roi')

In [2]:
# Initalize the dataset here
subj = 1
roi_label = 1
sess=2

roi_filename='lh.prf-visualrois.nii.gz'

roi_voxels = n.get_roi_voxels(subj, roi_label, roi_filename)

In [7]:
subj=1
batch_size=10

dset1 = n.nsd_dataset(subj, voxel_inds = roi_voxels)
trn1, val1, trn_subset1 = n.get_dataset_splits(dset1, subset_trn=True, rndseed = 983458)
valgenerator1 = n.nsd_dataloader(val1, batch_size=batch_size, shuffle=True, rndseed = 858875)

item1 = valgenerator1.__next__()

[278]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session02.hdf5
loading trial of interest
h5py loading time is 6.53
[12. 16. 11.  2. 19. 25. 23. 34. 34. 40.]
[714]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session11.hdf5
loading trial of interest
h5py loading time is 6.99
[12. 16. 11.  2. 19. 25. 23. 34. 34. 40.]
[165]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session12.hdf5
loading trial of interest
h5py loading time is 7.17
[12. 16. 11.  2. 19. 25. 23. 34. 34. 40.]
[154]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session16.hdf5
loading trial of interest
h5py loading time is 9.47
[12. 16. 11.  2. 19. 25. 23. 34. 34. 40.]
[471]
loading from /lab_data/tarrlab/common/data

In [11]:
dset2=nsd_dataset_alternatemethod(subj, voxel_inds = roi_voxels)
trn2, val2, trn_subset2 = n.get_dataset_splits(dset2, subset_trn=True, rndseed = 983458)
valgenerator2 = n.nsd_dataloader(val2, batch_size=batch_size, shuffle=True, rndseed = 858875)

item2 = valgenerator2.__next__()

In [ ]:
np.all(item1['data']==item2['data'])

In [13]:
# Testing the dataset splits/generator code

subj=1
batch_size=2
dset=n.nsd_dataset(subj, voxel_inds = roi_voxels)

trn, val, trn_subset = n.get_dataset_splits(dset, subset_trn=True, rndseed = 983458)
trngenerator = n.nsd_dataloader(trn, batch_size=batch_size, shuffle=False, rndseed = 983458)

trn_item = trngenerator.__next__()
trn_item = trngenerator.__next__()

trn_item['subject_df']

[686]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session26.hdf5
loading trial of interest
h5py loading time is 9.45
[35. 26.]
[34]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session35.hdf5
loading trial of interest
h5py loading time is 7.34
[35. 26.]
[632]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session06.hdf5
loading trial of interest
h5py loading time is 6.98
[11.  6.]
[545]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session11.hdf5
loading trial of interest
h5py loading time is 5.98
[11.  6.]


,trial_num,sess_num,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
8045,8046,11,35668,train2017,"(0.125, 0.125, 0, 0)",0.0,62212,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4382,4383,6,514988,train2017,"(0, 0, 0.125, 0.125)",0.0,60212,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [14]:
trnitem2 = trn[2:4]

trnitem2['subject_df']

[632]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session06.hdf5
loading trial of interest
h5py loading time is 6.10
[11.  6.]
[545]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session11.hdf5
loading trial of interest
h5py loading time is 6.15
[11.  6.]


,trial_num,sess_num,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
8045,8046,11,35668,train2017,"(0.125, 0.125, 0, 0)",0.0,62212,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4382,4383,6,514988,train2017,"(0, 0, 0.125, 0.125)",0.0,60212,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Testing the dataset splits/generator code

subj=1
batch_size=3
dset=n.nsd_dataset(subj, voxel_inds = roi_voxels)

trn, val, trn_subset = n.get_dataset_splits(dset, subset_trn=True, rndseed = 983458)
valgenerator = n.nsd_dataloader(val, batch_size=batch_size, shuffle=False)

val_item = valgenerator.__next__()
val_item = valgenerator.__next__()

val_item['subject_df']

[ 0 28 35]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.hdf5
loading whole file for this sess
h5py loading time is 16.38
[1. 1. 1.]
[44 55 56]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.hdf5
loading whole file for this sess
h5py loading time is 16.06
[1. 1. 1.]


,trial_num,sess_num,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
44,45,1,98480,train2017,"(0, 0, 0.125, 0.125)",0.076923,32625,False,True,True,...,8985,45,91,8985,45,91,8985,45,91,8985
55,56,1,440792,train2017,"(0, 0, 0.16640625, 0.16640625)",0.000000,53052,False,True,True,...,14898,56,354,14898,56,354,14898,56,354,14898
56,57,1,268114,train2017,"(0, 0, 0.125, 0.125)",0.000000,4930,False,True,True,...,13384,57,2829,13384,57,2829,13384,57,2829,13384


In [19]:
val_item2 = val[3:6]
val_item2['subject_df']

,trial_num,sess_num,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
44,45,1,98480,train2017,"(0, 0, 0.125, 0.125)",0.076923,32625,False,True,True,...,8985,45,91,8985,45,91,8985,45,91,8985
55,56,1,440792,train2017,"(0, 0, 0.16640625, 0.16640625)",0.000000,53052,False,True,True,...,14898,56,354,14898,56,354,14898,56,354,14898
56,57,1,268114,train2017,"(0, 0, 0.125, 0.125)",0.000000,4930,False,True,True,...,13384,57,2829,13384,57,2829,13384,57,2829,13384


In [21]:
# Testing the dataset splits/generator code

subj=1
batch_size=3
dset=n.nsd_dataset(subj, voxel_inds = roi_voxels)

trn, val, trn_subset = n.get_dataset_splits(dset, subset_trn=True, rndseed = 983458)
tsgenerator = n.nsd_dataloader(trn_subset, batch_size=batch_size, shuffle=True, rndseed = 858875)

ts_item = tsgenerator.__next__()
# ts_item = tsgenerator.__next__()

ts_item['subject_df']

[461]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session04.hdf5
loading trial of interest
h5py loading time is 7.07
[4. 8. 6.]
[622]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session06.hdf5
loading trial of interest
h5py loading time is 6.86
[4. 8. 6.]
[445]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session08.hdf5
loading trial of interest
h5py loading time is 7.92
[4. 8. 6.]


,trial_num,sess_num,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
2711,2712,4,24859,train2017,"(0, 0, 0.125, 0.125)",0.133333,64330,False,False,False,...,0,0,0,0,0,0,0,0,0,0
5695,5696,8,388203,train2017,"(0.1265625, 0.1265625, 0, 0)",0.000000,39643,False,False,False,...,0,0,0,0,0,0,0,0,0,0
4372,4373,6,138265,train2017,"(0, 0, 0.16328125, 0.16328125)",0.050000,63780,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [22]:
ts_item2 = trn_subset[0:3]

np.all(ts_item2['data']==ts_item['data'])

[416]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session09.hdf5
loading trial of interest
h5py loading time is 8.48
[ 9. 34. 17.]
[658]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session17.hdf5
loading trial of interest
h5py loading time is 9.49
[ 9. 34. 17.]
[472]
loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session34.hdf5
loading trial of interest
h5py loading time is 9.17
[ 9. 34. 17.]


False

In [12]:
subject_df = n.get_subj_stim_seq(1)
inds = np.arange(1498,1503,1)
subject_df.loc[1498:]

# np.sum(subject_df['sess_num']==1)

,trial_num,sess_num,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
1498,1499,2,406189,train2017,"(0, 0, 0.125, 0.125)",0.100000,44299,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1499,1500,2,479035,train2017,"(0, 0, 0.125, 0.125)",0.090909,62960,False,True,True,...,20474,1500,1575,20474,1500,1575,20474,1500,1575,20474
1500,1501,3,281323,train2017,"(0, 0, 0.1671875, 0.1671875)",0.000000,9227,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1501,1502,3,186308,train2017,"(0, 0, 0.16640625, 0.16640625)",0.000000,55016,False,False,False,...,0,0,0,0,0,0,0,0,0,0
1502,1503,3,468163,train2017,"(0, 0, 0.0, 0.0)",0.000000,60126,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# OLD CODE FOR ANOTHER WAY TO DO DATALOADING...SLOWER IT TURNS OUT

class nsd_dataset_alternatemethod(Dataset):
  
  """
  Custom dataset class that loads NSD data (voxels), and deals with corresponding image labels.
  Returns a Pytorch dataset object. Can access items with <object>[indices].
  When you get an item it is an object, which has fields 'data' (nTrials x nVoxels)
  and 'subject_df' which is a pandas dataframe listing everything about those trials. 
  """ 

  def __init__(self, subj, voxel_inds, trials_per_sess=750, num_sess = 40,  
               nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/',
               nsd_meta_file = '/lab_data/tarrlab/common/datasets/NSD/nsddata/experiments/nsd/nsd_stim_info_merged.pkl'):

    self.subj = subj;
    self.voxel_inds = voxel_inds;
    self.num_vox = np.shape(voxel_inds)[1]
   
    self.trials_per_sess=trials_per_sess
    self.num_sess=num_sess
    
    subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
    
    self.subj_data_dir = subj_data_dir
    self.nsd_meta_file = nsd_meta_file

  def __len__(self):

    if hasattr(self, 'data'):
        mylen = np.shape(self.data)[0]
    else:
        mylen = self.num_sess*self.trials_per_sess
        
    return mylen

  def __getitem__(self, trial_inds):

    if hasattr(trial_inds, '__len__')==True:
        nTrials = len(trial_inds)
    else:
        nTrials=1

    trial_inds = np.squeeze(np.array([trial_inds]))
    
    which_sess = np.floor(trial_inds/self.trials_per_sess)+1
    sess2load = np.unique(which_sess)
    data  = np.zeros([nTrials, self.num_vox])
    
    # looping over all the sessions we need data from
    for se in sess2load:
        
        trials_this_sess = (trial_inds[which_sess==se] - (se-1)*self.trials_per_sess).astype('int')
        t = time.time()
        # will need to sort these trial indices, because can't use non-increasing indices with the h5py selector.
        # using a custom function here to remember how to un-sort 
        order2sort, order2reverse = n.invertible_sort(trials_this_sess)
        trials_this_sess = trials_this_sess[order2sort]
        elapsed = time.time() - t
        print('sorting time elapsed is %.2f'%elapsed)
        
        print(trials_this_sess)
        # Load in the data for this session (slow)
        nii_fn = os.path.join(self.subj_data_dir,'betas_session%02d.hdf5'%se)
        print('loading from %s'%nii_fn)
        t = time.time()
        with h5py.File(nii_fn, "r") as f:
            data_this_sess = np.array(f['betas'][trials_this_sess,:,:,:])
        elapsed = time.time() - t
        print('h5py loading time is %.2f'%elapsed)
        
        t = time.time()
        # reverse the sorting, back to our desired order
        data_this_sess = data_this_sess[order2reverse,:,:,:]
        elapsed = time.time() - t
        print('reordering time is %.2f'%elapsed)
        
        # adjusting for the 300x scaling
        data_this_sess = data_this_sess/300
        
        # All NIFTI files in the prepared NSD data are in LPI ordering
        # (the first voxel is Left, Posterior, and Inferior)
        data_this_sess = data_this_sess[:,self.voxel_inds[0],self.voxel_inds[1],self.voxel_inds[2]]
        
        print(which_sess)
        # putting data from this session into the correct rows of our big matrix
        data[which_sess==se, :] = data_this_sess
        
    assert(np.shape(data)[0]==nTrials)

    df = n.get_subj_stim_seq(self.subj).loc[trial_inds.astype(int)]
                            
    item = {'data': data, 'subject_df': df}

    return item

In [6]:
## OLD CODE FOR ANOTHER WAY TO MAKE CUSTOM DATALOADER

class nsd_dataloader(Generator):
    
    def __init__(self, dset, batch_size, shuffle=True):
        
        super(nsd_dataloader, self).__init__(dset, batch_size, shuffle=True)
        self.collate_fn = self.__collate_fn__
        
    def __collate_fn__(self, batch):
        
        which_sess = [batch[ii]['which_sess'] for ]
        sess2load = np.unique(which_sess)
        data  = np.zeros([nTrials, self.num_vox])

        # looping over all the sessions we need data from
        for se in sess2load:

            trials_this_sess = (trial_inds[which_sess==se] - (se-1)*self.trials_per_sess).astype('int')

            # Load in the data for this session (slow)
            nii_fn = os.path.join(self.subj_data_dir,'betas_session%02d.hdf5'%se)
            print('loading from %s'%nii_fn)
            with h5py.File(nii_fn, "r") as f:
                data_this_sess = np.array(f['betas'][trials_this_sess,:,:,:])

            # adjusting for the 300x scaling
            data_this_sess = data_this_sess/300

            # All NIFTI files in the prepared NSD data are in LPI ordering
            # (the first voxel is Left, Posterior, and Inferior)
            data_this_sess = data_this_sess[:,self.voxel_inds[0],self.voxel_inds[1],self.voxel_inds[2]]

            # putting data from this session into the correct rows of our big matrix
            data[which_sess==se, :] = data_this_sess

        assert(np.shape(data)[0]==nTrials)

#         print(batch[0])
        return batch

SyntaxError: invalid syntax (<ipython-input-6-07f307e3ee6d>, line 12)

In [8]:
# Load in the data for this session (slow)
nii_fn = os.path.join(subj_data_dir,'betas_session%02d.hdf5'%sess)
print('loading from %s\n'%nii_fn)
with h5py.File(nii_fn, "r") as f:
    data = np.array(f['betas'])
        
data = data/300
print(np.shape(data))

loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj02/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session03.hdf5

(750, 84, 106, 82)


In [16]:
# Loading in the ROI file in vol space
roi_name = 'lh.prf-visualrois.nii.gz'
roi_fn = os.path.join(subj_roi_dir, roi_name)
roi_nii = nib.load(roi_fn)
roi_vol = roi_nii.get_fdata()
print(np.shape(roi_vol))
# want it to match dims of functional volume data
roi_vol = np.moveaxis(roi_vol, [0,1,2],[2,1,0])
print(np.shape(roi_vol))

(81, 104, 83)
(83, 104, 81)


In [19]:
# gather metadata for the NSD images
nsd_meta_file = '/lab_data/tarrlab/common/datasets/NSD/nsddata/experiments/nsd/nsd_stim_info_merged.pkl'
with open(nsd_meta_file,'rb') as f:
    stim_info = pickle.load(f,encoding="latin1")

In [6]:
# Figure out the labels of the COCO images included in this session for this subject

coco_ids = n.get_coco_ids_by_subject(subj, trial_inds=n.nsd_inds_each_sess(sess))
len(coco_ids)

750

In [9]:
# checking some basic properties of data

datmean = np.mean(data, axis=0)
datvar = np.std(data, axis=0)
np.mean(datmean)
np.mean(datvar)

1.3453987755388994

In [15]:
# test gathering the desired indices for each session

trial_inds = n.nsd_inds_each_sess(sess=[10,13], trials_per_sess=750)
print(trial_inds[750:760])
which_sess = np.floor(trial_inds/750)+1
sess2load = np.unique(which_sess)
sess2load

[9000. 9001. 9002. 9003. 9004. 9005. 9006. 9007. 9008. 9009.]


array([10., 13.])

In [13]:
# Test roi-voxel grabbing function

subj=3
roi_filename='lh.prf-visualrois.nii.gz'
roi_label=5
roi_voxels = n.get_roi_voxels(subj, roi_label, roi_filename)

roi_voxels

(array([23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25,
        25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
        25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        28, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
        29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30,
        30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 31,
        31, 

In [16]:
# Initalize the dataset

dset = n.nsd_dataset(subj, voxel_inds = voxel_inds)
trial_inds = n.nsd_inds_each_sess(sess=[10,11], trials_per_sess=750)
# trial_inds = trial_inds[0:10]

np.shape(dset[trial_inds]['data'])

loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj03/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session10.hdf5

loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj03/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session11.hdf5



(1500, 228)

In [18]:
# Testing time it takes to load data (long time)

nsd_dir = '/lab_data/tarrlab/common/datasets/NSD/'
subj_data_dir = os.path.join(nsd_dir,
                 'nsddata_betas','ppdata','subj%02d'%subj,'func1pt8mm',
                             'betas_fithrf_GLMdenoise_RR')
se=10
nii_fn = os.path.join(subj_data_dir,'betas_session%02d.hdf5'%se)

print('loading from %s\n'%nii_fn)
t = time.time()
with h5py.File(nii_fn, "r") as f:
    data_this_sess = np.array(f['betas'][0])
    

elapsed = time.time() - t
print('time elapsed is %.2f s'%elapsed)

print(np.shape(data_this_sess))

loading from /lab_data/tarrlab/common/datasets/NSD/nsddata_betas/ppdata/subj02/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session10.hdf5

time elapsed is 6.14 s
(84, 106, 82)


In [20]:
# Making sure these functions all agree

subj=1

nsd_inds = n.get_nsd_inds_by_subject(subj, trial_inds)

coco_inds =  n.get_coco_ids_by_subject(subj, trial_inds);

np.all(np.array(stim_info.loc[nsd_inds]['cocoId']) == coco_inds)

True

In [22]:
# Checking my functions for converting between nsd and coco indexing schemes

subj = 1
nsd_ind_orig = np.random.randint(0,73000, 6)
trial_inds = stim_info.loc[nsd_ind_orig]['subject1_rep0'] - 1 # MAKE SURE to switch from 1 to 0-indexing
trial_inds = trial_inds[trial_inds>-1]
nsd_inds = n.get_nsd_inds_by_subject(subj, trial_inds)
coco_ids =  n.get_coco_ids_by_subject(subj, trial_inds);

good_inds = np.array(stim_info.loc[nsd_ind_orig]['subject1_rep0']>0) | np.array(stim_info.loc[nsd_ind_orig]['subject1_rep1']>0) | np.array(stim_info.loc[nsd_ind_orig]['subject1_rep0']>0)

print(nsd_inds)
print(coco_ids)

stim_info.loc[nsd_ind_orig[good_inds]]

[17040]
[42553]


,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
17040,42553,train2017,"(0, 0.25, 0, 0)",0.0,17040,False,False,False,1,0,...,0,0,0,0,0,0,0,0,0,0
